In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [9]:
class CNN(nn.Module):
    def __init__(self,in_channels=1,num_classes=10):
        super(CNN,self).__init__()
        self.conv1=nn.Conv2d(in_channels=1,out_channels=8,kernel_size=(3,3),stride=(1,1),padding=(1,1))
        self.conv2=nn.Conv2d(in_channels=8,out_channels=16,kernel_size=(3,3),stride=(1,1),padding=(1,1))
        self.fc1=nn.Linear(16*7*7,128)
        self.fc2=nn.Linear(128,num_classes)
        self.pool=nn.MaxPool2d(kernel_size=(2,2),stride=(2,2))

    def forward(self,x):
        x=self.pool(F.relu(self.conv1(x)))
        x=self.pool(F.relu(self.conv2(x)))
        x=x.view(x.size(0),-1)
        x=F.relu(self.fc1(x))
        x=self.fc2(x)
        return x
print(CNN())

CNN(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
)


In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

input_size=28*28
num_classes=10
learning_rate=0.001
batch_size=64
num_epochs=5
load_model=True
train_dataset=datasets.MNIST(root='dataset',train=True,transform=transforms.ToTensor(),download=True)
train_loader=DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_dataset=datasets.MNIST(root='dataset',train=False,transform=transforms.ToTensor(),download=True)
test_loader=DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)

In [11]:
model=CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=learning_rate)

In [12]:
def save_checkpoint(state,filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state,filename)
def load_checkpoint(checkpoint):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])


In [16]:
if load_model:
    checkpoint=torch.load("my_checkpoint.pth.tar", weights_only=True)
    load_checkpoint(checkpoint)

=> Loading checkpoint


In [17]:
#training the model
for epoch in range(num_epochs):
    losses=[]
    if epoch%2==0:
        checkpoint={'state_dict': model.state_dict(),'optimizer': optimizer.state_dict()}
        save_checkpoint(checkpoint)

    for batch_idx,(data,targets) in enumerate(train_loader):
        data=data.to(device=device)
        targets=targets.to(device=device)
        scores=model(data)
        loss=criterion(scores,targets)
        losses.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    mean_loss=sum(losses)/len(losses)

    print(f'epoch {epoch+1}/{num_epochs},loss={mean_loss:.4f}')


=> Saving checkpoint
epoch 1/5,loss=0.0199
epoch 2/5,loss=0.0176
=> Saving checkpoint
epoch 3/5,loss=0.0133
epoch 4/5,loss=0.0117
=> Saving checkpoint
epoch 5/5,loss=0.0107


In [15]:
def check_accuracy(loader,model):
    num_correct=0
    num_samples=0
    model.eval()
    with torch.no_grad():
        for x,y in loader:
            x=x.to(device=device)
            y=y.to(device=device)

            scores=model(x)
            _,predictions=scores.max(1)
            num_correct+=(predictions==y).sum()
            num_samples+=predictions.size(0)
    print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')
    model.train()

check_accuracy(train_loader,model)
check_accuracy(test_loader,model)

Got 59709 / 60000 with accuracy 99.52
Got 9872 / 10000 with accuracy 98.72
